# Using Tecton to provide context for LangChain applications 

This tutorial guides you through creating a restaurant recommendation AI function using LangChain within a consumer restaurant rating app. The app allows users to:

- Record restaurants they visit.
- Rate their dining experiences.

Tecton collects user activity data from rating events to build a profile of a user's tastes by building an feature that lists:

- Preferred cuisines.
- Average star ratings given per cuisine.
- Total restaurant visits per cuisine.

A feature service that serves this feature on-demand for a given user has also been implemented in this sample Tecton workspace and will be used to enrich a LangChain prompt.  
In this tutorial you will: 
- Test a langchain prompt without user data enrichment.
- Enrich a LangChain prompt to provide a personalized recommendation.

## Install Packages

In [ ]:
# install langchain components
!pip install langchain-openai -q
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain_core -q
!pip install openai -q
!pip install tecton_client -q

## Build a chain 

Obtain an LLM model to use.

The following cell instantiates a LangChain model using OpenAI's GPT-4o-mini. 

For this step you will need to:
- Obtain an [OpenAI API key](https://platform.openai.com/api-keys) and replace "your-openai-key" in the following cell.


In [ ]:
import os
from langchain_openai import ChatOpenAI

# replace with your key
os.environ["OPENAI_API_KEY"] = "your-openai-key"

# instantiate chat model
model = ChatOpenAI(model="gpt-4o-mini")

The next cell shows a prompt without any personalization. Since it does not have information about the user, it only provides a general recommendation.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """You are a consierge service that recommends restaurants. 
        Respond to the user query about dining. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know, and suggested menu items. 
        User query:{user_query}"""
)

chain = prompt | model | StrOutputParser()

# test the prompt
inputs = {
    "user_query": "suggest a restaurant for tonight in Ballantyne area of Charlotte and tell me why you suggest it"
}
chain.invoke(inputs).splitlines()

Your results may vary, the recommendation for me was:

    I recommend trying **The Capital Grille** in the Ballantyne area of Charlotte. 
    The address is **7830 Gateway Village Blvd, Charlotte, NC 28277**.',

    The Capital Grille is an upscale steakhouse known for its dry-aged steaks and an extensive wine list. The ambiance is elegant and perfect for a nice dinner out. 

    I suggest trying the **Bone-In Ribeye** or the **Filet Mignon**, both cooked to perfection. For a delicious side, the **Truffle Fries** or **Lobster Mac 'n' Cheese** are highly recommended. If you're in the mood for something lighter, their **Wedge Salad** is a refreshing choice.

    Overall, it’s a great place for a special occasion or a memorable dining experience. Enjoy your evening!

Not bad, it provided a good restaurant and a reason it is good in a general sense, but now let's personalize it to the user's tastes based on their rating activity.

## Adding the user's tastes to the prompt

In the following cell you change the prompt to include the user's tastes `{cuisines}` and provide instructions for the LLM on how to use that data.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

personalized_prompt = ChatPromptTemplate.from_template(
    """You are a consierge service that recommends restaurants. 
        Respond to the user query about dining. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Respond to the user query by taking into account the user's dining history. 
        Show their rating of the cuisine you recommend.
        If the user does not provide a cuisine, choose a restaurant that fits a cuisine from their highest average ratings:
        User's dining history by cuisine: {cuisines}
        User query:{user_query}"""
)

personalized_chain = personalized_prompt | model | StrOutputParser()

### Use Tecton to get the latest context

We'll access a Tecton Feature Service to get the user's latest tastes as an array of cuisines. Below is the view of the whole data pipeline that processes `user` and `restaurant` data to produce the `cuisine_service` features you are going to use:

![](./cuisines_pipeline.png)

`user`, `restaurant` and `ratings` data are used to calculate the latest features creating a user profile, restaurant profile and the user's taste profile. In this example we use the `cuisines_service` to access the user's tastes as an average rating by cuisine over a 5 year window.

Tecton offers [multiple APIs to access features](https://docs.tecton.ai/docs/reading-feature-data/reading-feature-data-for-inference). In the cell below you will use the [Python client](https://docs.tecton.ai/docs/reading-feature-data/reading-feature-data-for-inference/reading-online-features-for-inference-using-the-python-client) to interact with Tecton's feature serving engine. 

You'll need a Tecton API key to replace "your-api-key" below.

PROBABLY NEED TO ADJUST THESE INSTRUCTIONS ONCE WE PUT THE TUTORIAL ON `explore.tecton.ai`.
 
To obtain a Tecton API key this, you will first need to create a new Service Account and give it access to read features from your workspace.

✅ Head to the following URL to create a new service account (replace "explore" with your organization's account name in the URL as necessary). Be sure to save the API key!

https://community.tecton.ai/app/settings/accounts-and-access/service-accounts?create-service-account=true

✅ If you are using explore.tecton.ai, this account will automatically be given the necessary privileges to read features from the "prod" workspace. Otherwise, you should give the service account access to read features from your newly created workspace by following these steps:



In [ ]:
from tecton_client import TectonClient

tecton_client = TectonClient(
    url="https://community.tecton.ai/",
    api_key="your-tecton-api-key",  # REPLACE WITH YOUR TECTON API KEY
    default_workspace_name="gen-ai-prompt",
)


In order to retrieve a particular user's information, we'll need their `user_id`. 

In the next cell you:
- Use a hard-coded value for `user_id` to simulate the user's session.
- Retrieve the user's data from the `cuisines_service` Feature Service using `get_features`.

In [ ]:
# retrieve features for "current" user
user_id = "a6afb498-b24f-4314-93df-5a5040cf1cb7"


features = tecton_client.get_features(
    feature_service_name="cuisines_service", join_key_map={"user_id": user_id}
).get_features_dict()

for f in features["user_ratings_and_total_visits_by_cuisine.user_ratings_and_total_visits_by_cuisine"]:
    print(f)

### Putting it together with the chain
In the following cell, you create a recommendation function that:
- Retrieves the user's tastes feature and adds it to the LangChain inputs as the `cuisines`
- Invokes the LLM using the now personalized_chain and the user query

In [ ]:
def get_personalized_recommendation(langchain_chain, user_id, user_query):
    # retrieve user features from Tecton
    features = tecton_client.get_features(
        feature_service_name="cuisines_service", join_key_map={"user_id": user_id}
    ).get_features_dict()

    # build inputs for chat
    inputs = {
        "cuisines": features["user_ratings_and_total_visits_by_cuisine.user_ratings_and_total_visits_by_cuisine"],
        "user_query": user_query,
    }

    # run the chain with inputs
    return langchain_chain.invoke(inputs)


# test the function
get_personalized_recommendation(
    personalized_chain,
    user_id,
    user_query="suggest a restaurant for tonight in Ballantyne area of Charlotte and tell me why you suggest it",
).splitlines()

##

The output at the time of writing this was:

    For a delightful dining experience tonight in the Ballantyne area of Charlotte, I recommend **The Cowfish Sushi Burger Bar**. ',

    **Address:** 4720 Holly Crest Ln, Charlotte, NC 28277',
    This restaurant offers a unique fusion of American and Japanese cuisines, 
    which aligns perfectly with your high average ratings in American (4.02) 
    and your interest in Asian Fusion (3.95). ',


    **Suggested Menu Items:**
    - **Burgushi Rolls**: A fun combination of sushi and burgers, perfect for a twist on traditional dishes.
    - **The Cowfish Burger**: A classic American burger with a variety of toppings to choose from.
    - **Lobster and Shrimp Roll**: A delicious sushi option for seafood lovers.
    - **Signature Milkshakes**: Don''t miss out on their creative milkshakes for dessert!

    I believe you''ll enjoy the vibrant atmosphere and the innovative menu that combines two of your favorite cuisines!

It makes a big difference when we are able to provide fresh context to the LLM. It not only "considered" the cuisines for the specific user, but it found an option that combines 2 of the user's favorites. Prompt engineering leads to better  instructions for the LLM. Combining the prompt with fresh and relevant data improves of the LLM's response. 

LangChain provides a great framework on which to build LLM based applications. 

Tecton enhances the chain by providing relevant information and up to date information for real-time, streaming and batch data. This application could be further personalized by taking into account more user information in the prompt. For example, the user's profile could include their dietary restrictions and only recommend restaurants that have options for them. The app could use their current location and local time to automatically suggest restaurant that are near and suggest a breakfast, lunch or dinner experience based on the time of day. Tecton's ability to transform batch, streaming and real-time data delivering the latest feature values to the app, is a great way of giving your generative AI application the context it needs.